<a href="https://colab.research.google.com/github/mahika112/AI-Agents/blob/main/Mahika_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [59]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [60]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [61]:
os.environ['GOOGLE_API_KEY']= getpass.getpass("Enter the Gemini API Key")

Enter the Gemini API Key··········


# STEP 2: Initialize Gemini 1.5 Flash

In [62]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-1.5-flash-latest", temperature=0.2)

# STEP 3: Node to ask user for symptom

In [63]:
def get_symptom(state: dict) ->dict:
  symptom = input("Welcome to XYZ hospital, Please enter your symptom")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [64]:
def classify_symptom(state: dict) -> dict:
  prompt = (
      "You are a helpful MEdical Assistant, Classify the symptom below into one of the categories \n"
      "-General\n -Emergency\n -mental\n"
      f"Sympton: {state['symptom']} \n"
      "Respond only with one word: General, Emergency, Mental Health"
      "#Example : input: I have fever, Output : General"
  )
  response = llm.invoke([HumanMessage(content=prompt) ])
  category = response.content.strip()
  print(f"LLM classifies the symptom as :{category}") #debug
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [65]:
def symptom_router(state: dict) ->dict:
  cat = state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency" in cat:
    return "emergency"
  elif "mental" in cat:
    return "mental_health"
  else:
      return "general"

# STEP 6: Category-specific response nodes


In [66]:
def general_node(state: dict) ->dict:
  state["answer"] = f" '{state['symptom']}' : seems general : directing you to general ward for consulting a doctor"
  return state

def emergency_node(state: dict) ->dict:
  state["answer"] = f" '{state['symptom']}' : It is a Medical Emergency : seeking imeediate help"
  return state

def mental_health_node(state: dict) ->dict:
  state["answer"] = f" '{state['symptom']}' seems a medical health issue: talk to our consultant"
  return state


# STEP 7: Build LangGraph


In [67]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router, {
    "general" : "general",
    "emergency" : "emergency",
    "mental_health" : "mental_health"

})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)

# STEP 8: Compile and invoke the graph


In [68]:
graph = builder.compile()

In [70]:
final_state = graph.invoke({})
print("final Output \n")
print(final_state["answer"])


Welcome to XYZ hospital, Please enter your symptomI m suffering from high depression
LLM classifies the symptom as :Mental
final Output 

 'I m suffering from high depression' seems a medical health issue: talk to our consultant
